In [20]:
%load_ext IPython.extensions.autoreload
%autoreload 2

The IPython.extensions.autoreload extension is already loaded. To reload it, use:
  %reload_ext IPython.extensions.autoreload


In [32]:
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import datetime as dt

from data import FinData
from preprocessing import train_valid_split
from model import CatboostFinModel

In [38]:
data = FinData("../datasets/T_yandex_10min.csv")

data.insert_all()
data.make_binary_class_target()

In [39]:
data.cat_features

['hours', 'day', 'minute', 'is_holiday', 'pred_holiday', 'week_pred_holiday']

In [40]:
numeric = data.numeric_features[::5] + data.numeric_features[1::5]

cat = data.cat_features

args = {"iterations" : 10000, 
        "depth" : 5, 
        "learning_rate" : 0.01, # тут слегка неадекватные параметры, которые можно менять 
        "use_best_model" : True, 
        "l2_leaf_reg" : 200,
        "verbose" : 100, # отчет каждые 100 итераций 
        "loss_function" : 'CrossEntropy', 
        "eval_metric" : 'Accuracy', 
        "cat_features" : cat, 
        "random_state" : 42,
        "early_stopping_rounds" : 5000}

# data.restrict_time_down(2024, 11, 1)

In [41]:
X_train, X_val, y_train, y_val = train_valid_split(data=data.df, 
                                                   year=2024, month=12, day=5, 
                                                   numeric=numeric, cat=cat, target="direction_binary")

In [42]:
model = CatboostFinModel(args)

model.set_datasets(X_train, X_val, y_train, y_val)
model.set_features(numeric, cat)

model.fit()

0:	learn: 0.5578580	test: 0.6111111	best: 0.6111111 (0)	total: 318ms	remaining: 53m 2s
100:	learn: 0.5591914	test: 0.6111111	best: 0.6161616 (17)	total: 10.9s	remaining: 17m 52s
200:	learn: 0.5600554	test: 0.6111111	best: 0.6161616 (17)	total: 20.7s	remaining: 16m 51s
300:	learn: 0.5608971	test: 0.6111111	best: 0.6161616 (17)	total: 31.4s	remaining: 16m 51s
400:	learn: 0.5621560	test: 0.6111111	best: 0.6161616 (17)	total: 42.5s	remaining: 16m 57s
500:	learn: 0.5634744	test: 0.6060606	best: 0.6161616 (17)	total: 54s	remaining: 17m 4s
600:	learn: 0.5643086	test: 0.6010101	best: 0.6161616 (17)	total: 1m 7s	remaining: 17m 35s


KeyboardInterrupt: 